In [ ]:
# === Experiment: Hash partitioning for GroupBy(User_ID) -> avg(Heart_Rate) ===
# Produces: /data/exp_hash_user_avg_results.csv

from pyspark.sql import SparkSession, functions as F
from pyspark.storagelevel import StorageLevel
import os, time
import pandas as pd

# ---------- Config ----------
SOURCE_CSV  = "file:///data/cleaned_personal_health_data.csv"   # adjust if different
OUTPUT_CSV  = "/data/exp_hash_user_avg_results.csv"
PARTITIONS_LIST = [2, 4, 8, 16, 32]   # change if you want more/fewer
RUNS_PER_SETTING = 5                  # repeats per partition count for stability
LIMIT_USERS = int(os.environ.get("LIMIT_USERS", "0"))  # optional: subset for speed (0 = no limit)
SPARK_SHUFFLE_PARTS = 4               # keep constant for fairness

# ---------- Spark session (reuse existing if present) ----------
spark = SparkSession.builder \
    .appName("Exp-Hash-UserAvg") \
    .config("spark.sql.shuffle.partitions", str(SPARK_SHUFFLE_PARTS)) \
    .config("spark.speculation", "false") \
    .getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# ---------- Load & prep ----------
df = spark.read.csv(SOURCE_CSV, header=True, inferSchema=True)
df = df.select("User_ID", "Heart_Rate")  # only what we need

# Optional: restrict to a subset of users (useful on slow disks / large data)
if LIMIT_USERS > 0:
    subset = df.select("User_ID").distinct().limit(LIMIT_USERS)
    df = df.join(subset, on="User_ID", how="inner")
    print(f"[INFO] LIMIT_USERS={LIMIT_USERS} -> rows after filter: {df.count()}")

# Warm-up read/parse
_ = df.limit(1).count()

results_rows = []

for P in PARTITIONS_LIST:
    # Hash partitioning on the key
    base = df.repartition(P, "User_ID").persist(StorageLevel.MEMORY_AND_DISK)
    # Materialize cache to remove first-run noise from timings
    _ = base.count()

    # One not-timed warm-up of the exact query
    _ = base.groupBy("User_ID").agg(F.avg("Heart_Rate").alias("Avg_Heart_Rate")).count()

    times = []
    for r in range(1, RUNS_PER_SETTING + 1):
        t0 = time.perf_counter()
        # Action: triggers full shuffle+agg
        _ = base.groupBy("User_ID").agg(F.avg("Heart_Rate").alias("Avg_Heart_Rate")).count()
        elapsed = round(time.perf_counter() - t0, 6)
        times.append(elapsed)
        print(f"[hash] P={P:>2} run {r}/{RUNS_PER_SETTING}: {elapsed:.6f}s")

    avg_t = round(sum(times) / len(times), 6)
    results_rows.append({
        "Partitioning Type": "hash",
        "Partitions": P,
        "Runs": RUNS_PER_SETTING,
        "Per-Run Times (s)": ";".join(map(str, times)),
        "Avg Execution Time (s)": avg_t,
        "LimitUsers": LIMIT_USERS
    })
    # free cache for this setting
    base.unpersist()

# ---------- Save CSV ----------
pdf = pd.DataFrame(results_rows)
pdf.to_csv(OUTPUT_CSV, index=False)
print(f"\n[OK] Wrote {OUTPUT_CSV}")
display(pdf)
